<a href="https://colab.research.google.com/github/deepanshgoyal33/DC_GAN/blob/master/DC_gan_implementation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [5]:
#for Reproducibility
manual_seed = 100
random.seed(manual_seed)
torch.manual_seed(manual_seed)

In [6]:
#Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#give the path, where Dataste is kept
path = "/content/drive/My Drive/Celeb_A_Dataset/DC_GAN"

In [17]:
#Initialising the parameters
dataroot = path
workers = 2
batch_size = 128
image_size = 64
n_channels = 3
nz = 100
#size = n, generator =g, discriminator=f, features = f
ngf = 64
ndf = 64
n_epochs = 100
lr = .002
beta1 = 0.5
ngpu = torch.cuda.device_count()
print(ngpu)

1


In [0]:
#Dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# cuda dependencies and Gpu mounting
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


In [0]:

batch = next(iter(dataloader))

In [0]:
 #Function to initialise weights for a layer
 def weights_init(m):
   classname = m.__class__.__name__
   if classname.find('Conv')!= -1:
     nn.init.normal_(m.weight,0.0,0.02)
   elif classname.find('BatchNorm') != -1:
     nn.init.normal_(m.weight.data,1.0,.02)
     nn.init.constant_(m.bias.data,0)

In [0]:
#Making the Generator Module and its architecture
class Generator(nn.Module):
  def __init__(self, ngpu):
    super(Generator, self).__init__()
    self.ngpu = ngpu
    self.main = nn.Sequential(
        # input is Z, going into a convolution
        nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
        nn.BatchNorm2d(ngf * 8),
        nn.ReLU(True),
        # state size. (ngf*8) x 4 x 4
        nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf * 4),
        nn.ReLU(True),
        # state size. (ngf*4) x 8 x 8
        nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf * 2),
        nn.ReLU(True),
        # state size. (ngf*2) x 16 x 16
        nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf),
        nn.ReLU(True),
        # state size. (ngf) x 32 x 32
        nn.ConvTranspose2d( ngf, n_channels, 4, 2, 1, bias=False),
        nn.Tanh()
        # state size. (nc) x 64 x 64
        )
  def forward(self, input):
        return self.main(input)

In [22]:
#initialing the model with a argument
generatornet = Generator(ngpu).to(device)
generatornet.apply(weights_init)
print(generatornet)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [0]:
# Architecture of the Discriminator
## Inorder to chcek each layer feature padding,batch norm and bias structre you 
##  can use the model_dict or state_dict function
class Discriminator(nn.Module):
  def __init__(self,ngpu):
    super(Discriminator ,self).__init__()
    self.ngpu= ngpu
    self.main = nn.Sequential(
        nn.Conv2d(n_channels, ndf,4,2,1,bias= False),
        nn.LeakyReLU(0.2,inplace=True),
        nn.Conv2d(ndf,ndf*2,4,2,1,bias = True),
        nn.BatchNorm2d(ndf*2),
        nn.LeakyReLU(0.2,inplace = True),
        nn.Conv2d(ndf*2,ndf*4,4,2,1,bias = True),
        nn.BatchNorm2d(ndf*4),
        nn.LeakyReLU(0.2,inplace = True),
        nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ndf * 8),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(ndf*8,1,4,1,0,bias=False),
        nn.Sigmoid()      
    )
  def forward(self, input):
      return self.main(input)

In [24]:
#Initialising the discriminatornet
discriminatornet = Discriminator(ngpu).to(device)
discriminatornet.apply(weights_init)
print( discriminatornet)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [0]:
# All the optimizers, loss function and noise defined
criterion = nn.BCELoss()
fixed_noise = torch.randn(64,nz,1,1,device=device)
real_label = 1
fake_label =0
optimizerG = optim.Adam(generatornet.parameters(),lr =lr,betas = (beta1,0.999))
optimizerD = optim.Adam(discriminatornet.parameters(),lr =lr,betas = (beta1,0.999))

In [0]:
## Variables to store the in-training values
img_list = []
G_losses = []
D_losses = []
iters = 0

In [0]:
#Training Loop
print("Starting Training Loop..")
for epoch in range(n_epochs):
  for i, data in enumerate(dataloader,0):
    #clearing memory
    discriminatornet.zero_grad()
    #batch formatting
    real_cpu = data[0].to(device)
    b_size = real_cpu.size(0)
    #real samples
    label = torch.full((batch_size,), real_label, device=device)
    #output values using the real batch by forward passing through the discrimintor
    output = discriminatornet(real_cpu)
    output = output.view(-1)
    error_disReal = criterion(output,label)
    error_disReal.backward()
    D_x = output.mean().item()
    
    ##fake batch 
    noise = torch.randn(batch_size,nz,1,1,device= device)
    fake = generatornet(noise)
    label.fill_(fake_label)
    output = discriminatornet( fake.detach()).view(-1)
    error_disFake = criterion( output,label)
    error_disFake.backward()
    D_G_z1 = output.mean().item()
    
    error_dis = error_disFake +error_disReal
    optimizerD.step()


    #################Generator Network
    discriminatornet.zero_grad()
    label.fill_(real_label)
    output = discriminatornet(fake).view(-1)
    error_gen = criterion(output,label)
    error_gen.backward()
    D_G_z2 = output.mean().item()
    optimizerG.step()


    if i % 50 == 0:
      print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, n_epochs, i, len(dataloader),
                     error_dis.item(), error_gen.item(), D_x, D_G_z1, D_G_z2))
    G_losses.append(error_gen.item())
    D_losses.append(error_dis.item())
    if (iters % 500 == 0) or ((epoch == n_epochs-1) and (i == len(dataloader)-1)):
      with torch.no_grad():
        fake = generatornet(fixed_noise).detach().cpu()
        img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

    iters += 1